In [1]:
#Libs necessárias
import utils
from utils import *
import spacy
import PyPDF2
import pandas as pd
from time import time 
import re
import unidecode
import numpy as np
import os
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from calendar import month_name
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load csv file containing labeled economic news
#df = pd.read_csv('Full-Economic-News-DFE-839861.csv',encoding='latin1')
df = pd.read_csv('us-economic-newspaper.csv',encoding='latin1')
print(df.shape)
df.head()

(5015, 19)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,orig__golden,articleid,date,headline,lineid,next_sentence,positivity_gold,previous_sentence,relevance_gold,text
0,729487630,False,finalized,3,6/8/15 14:26,NaN,0.0000,not sure,0.3469,NaN,109092213,2/23/93,Nasdaq Index Falls 1.7% But Dow Stocks Are Up:...,109092213_01,"The Nasdaq composite index, home of technology...",NaN,NaN,NaN,The stock market accelerated its screeching sw...
1,729487631,False,finalized,3,6/11/15 10:58,6.0,0.3675,yes,1.0000,NaN,109092213,2/23/93,Nasdaq Index Falls 1.7% But Dow Stocks Are Up:...,109092213_02,"The bond market continued to rally, propelling...",NaN,The stock market accelerated its screeching sw...,NaN,"The Nasdaq composite index, home of technology..."
2,729487632,False,finalized,3,6/6/15 0:15,5.0,0.3416,yes,0.6771,NaN,109092213,2/23/93,Nasdaq Index Falls 1.7% But Dow Stocks Are Up:...,109092213_03,The Nasdaq market was stricken by the collapse...,NaN,"The Nasdaq composite index, home of technology...",NaN,"The bond market continued to rally, propelling..."
3,729487633,False,finalized,3,6/14/15 20:27,4.0,0.6756,yes,1.0000,NaN,109092213,2/23/93,Nasdaq Index Falls 1.7% But Dow Stocks Are Up:...,109092213_04,uring the marketÛªs split ÛÓ was 4.26 percen...,NaN,"The bond market continued to rally, propelling...",NaN,The Nasdaq market was stricken by the collapse...
4,729487634,False,finalized,3,6/6/15 13:22,3.0,0.6509,yes,1.0000,NaN,109092213,2/23/93,Nasdaq Index Falls 1.7% But Dow Stocks Are Up:...,109092213_05,". ""It's a nervous market,Û said Lawrence R. ...",NaN,The Nasdaq market was stricken by the collapse...,NaN,uring the marketÛªs split ÛÓ was 4.26 percen...


In [3]:
#Checking categories
df['positivity'].value_counts()

4.0    814
6.0    548
3.0    541
5.0    364
7.0    249
1.0    155
2.0    149
8.0     65
9.0     14
Name: positivity, dtype: int64

In [4]:
#Checking NaN values
df.isnull().sum()

_unit_id                    0
_golden                     0
_unit_state                 0
_trusted_judgments          0
_last_judgment_at          40
positivity               2116
positivity:confidence     942
relevance                   8
relevance:confidence        8
orig__golden             4998
articleid                   0
date                        0
headline                    0
lineid                      0
next_sentence               2
positivity_gold          4980
previous_sentence        1002
relevance_gold           4958
text                        0
dtype: int64

In [5]:
#Remove empty columns
df.drop(['positivity:confidence','positivity_gold','positivity_gold','relevance_gold','_last_judgment_at',
         'relevance','relevance:confidence','orig__golden','next_sentence','previous_sentence'],axis=1,inplace=True)
df.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       'positivity', 'articleid', 'date', 'headline', 'lineid', 'text'],
      dtype='object')

In [6]:
#Remove empty columns
#df.drop(['positivity:confidence','positivity_gold','positivity_gold','relevance_gold'],axis=1,inplace=True)
#df.columns

In [7]:
#Separate categories into 0 or 1
#0 is negative and 1 is positive - obtain more balanced dataset
#lista_class = []

#for i in df['positivity']:
#    if i < 6:
#        lista_class+=[0] #negative
#    else:
#        lista_class+=[1] #positive
#df['class'] = pd.DataFrame(lista_class)

In [8]:
#Separate categories into 0 or 1
#0 is negative and 1 is positive - obtain more balanced dataset
lista_class = []

for i in df['positivity']:
    if i < 5:
        lista_class+=[0] #negative
    else:
        lista_class+=[1] #positive
df['class'] = pd.DataFrame(lista_class) 

In [9]:
#Checking new classes
df['class'].value_counts()

1    3356
0    1659
Name: class, dtype: int64

In [10]:
#Remove Nan
df=df.dropna()
print(df.shape)
df.isnull().sum()

(2899, 11)


_unit_id              0
_golden               0
_unit_state           0
_trusted_judgments    0
positivity            0
articleid             0
date                  0
headline              0
lineid                0
text                  0
class                 0
dtype: int64

In [11]:
#Cleaning
d = df.copy()
d['text'] = d['text'].apply(lambda x:x.lower())
d['text'] = d['text'].str.replace('</br>',' ')
d['text'] = d['text'].str.replace('-',' ')

#Remove ponctuation
exclude = set(string.punctuation)
d['text'] = d['text'].apply(lambda texto:''.join(ch for ch in texto if ch not in exclude))
d['text'] = d['text'].apply(lambda x:clean_ents(x))
d['text'] = d['text'].apply(lambda x:sub_str(x))
d['text'] = d['text'].apply(lambda x:clean_words(x))
d = d.reset_index(drop=True)

In [12]:
#Count most frequent words in dataset
Counter(" ".join(d["text"]).split()).most_common(30)

[('market', 647),
 ('rates', 558),
 ('rate', 527),
 ('interest', 471),
 ('prices', 439),
 ('inflation', 410),
 ('economic', 388),
 ('economy', 380),
 ('money', 252),
 ('unemployment', 233),
 ('higher', 228),
 ('price', 226),
 ('increase', 222),
 ('average', 215),
 ('growth', 214),
 ('recession', 207),
 ('rose', 200),
 ('decline', 186),
 ('tax', 181),
 ('rise', 180),
 ('term', 174),
 ('index', 169),
 ('business', 162),
 ('high', 160),
 ('long', 158),
 ('credit', 158),
 ('markets', 152),
 ('fell', 146),
 ('industrial', 146),
 ('bonds', 145)]

In [13]:
#Machine Learning Algo
#Features para criação do modelo preditivo

features = d[['articleid','text']]

labels = d['class']

print('Tamanho da base de features:',features.shape)

print('Tamanho da base de labels:',labels.shape)

Tamanho da base de features: (2899, 2)
Tamanho da base de labels: (2899,)


In [14]:
#Pacotes para Vetorização dos Textos e Machine Learning (varios testes para modelos distintos)

import sklearn

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score

from sklearn import metrics

from sklearn.metrics import classification_report

from sklearn.pipeline import make_pipeline

from sklearn import model_selection

from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import fbeta_score, make_scorer

from sklearn.metrics import precision_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import recall_score

from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import cross_val_score

 

In [15]:
#Split em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features,labels, test_size=0.33, random_state=42)

print(X_train.shape)

print(y_train.shape)

(1942, 2)
(1942,)


In [16]:
#Adding more stopwords
stopw += ['said','president','wall','street','standard','poors','ll','ve','vice','dow','jones','nasdaq','th',
          'chairman', 'chief', 'executive','new','york','analysts','federal','investors','economist','government',
          'economists','dollar','point','stock','time','stocks','points','bank','bond','nations','traders','face',
          'times','yields','generally','data','meeting','committee','figures','director','chairman','going','home','copom',
          'session','people','public','according','blue','chip','feds','banks','area','city','says','officials','day','secretary',
          'report','showed','todays','health','middle','class','crude','oil','focus','survey','ex','ante']


In [17]:
#Pipeline com aplicação do Modelo Preditivo - LogReg

pipeline = Pipeline([

    ('tfidf', TfidfVectorizer(stop_words=stopw,strip_accents='unicode',ngram_range=(2,4),

    max_features=300)),

    ('clf',LogisticRegression(penalty='l1', C = 0.55))])

 

#Média e desvio padrão do modelo usando cross-validation

score_nb=cross_val_score(pipeline,X_train['text'],y_train,cv=StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42),scoring='f1_micro')

print(score_nb.mean())

print(score_nb.std())



0.58003120124805
0.0018193297643823029


In [18]:
# Fit do pipeline

pipeline.fit(X_train['text'], y_train)

from sklearn.metrics import f1_score

print(f1_score(y_test, pipeline.predict(X_test['text']), average = 'micro'))

0.5590386624869383


In [19]:
#Checking LR coefs
coefs = pd.DataFrame({"coef":pipeline['clf'].coef_[0],"word":pipeline['tfidf'].get_feature_names()}).sort_values("coef", 
ascending=False)#.query("coef!=0")
coefs

,coef,word
111,1.091135,index rose
201,0.636293,prime rate
285,0.518250,total return
112,0.391397,industrial average
171,0.381318,mutual fund
...,...,...
217,-0.149100,rate inflation
9,-0.161947,bad news
42,-0.267513,credit card
110,-0.351935,index fell


In [20]:
import collections

#Adding words and phrases that are going to be relevant for BCB texts, not exactly for the database used to train the algo

vocabulary={'growth slowdown':300,
            'causing significant slowdown':301,
         'remains challenging':302,
         'sharp drop':303,
         'historically low levels':304,
         'supply shock':305,
         'economic recovery delayed':306,
         'remains uncertain': 307,
         'economic contraction':308,
         'uncertainty arises':309,
         'favorably expected signs':310,
         'consistent stabilization':311,
         'gradual recovery':312,
         'favorable global economic activity':313,
         'consistent recovery':314,
         'prospects economic recovery':315,
         'anchored expectations':316,
         'frustration expectations':317,
         'evolved favorably':318,
         'benign evolution':319,
         'gradual normalization':320,
         'levels comfortable':321,
         'conditions favorable':322,
         'easing process continue':323,
         'employment gains':324, 'recovery advances':325, 'favorable evolution':326, 'consumption growth':327,
         'stronger recovery':328,'evolved expected':329,'lower global growth':330,'benign behavior':331, 'comfortable levels':332
    
}

 
#Creating new vocab for TFIDF
d=pipeline['tfidf'].vocabulary_

od = collections.OrderedDict(sorted(d.items(),reverse=False,key=lambda x:x[0]))

n=len(od)

for feature, index in vocabulary.items():

    if feature not in od.values():

        od[feature] = n + index

#Final vocab

final_vocab = {key: v for key,v in zip(od,range(0,len(od)))}

In [21]:
#Inputing random weights for the words added above
dict_weights = {
            'growth slowdown':-0.5,
            'causing significant slowdown':-0.5,
         'remains challenging':-0.2,
         'sharp drop':-0.7,
         'historically low levels':-1,
         'supply shock':-0.5,
         'economic recovery delayed':-0.5,
         'remains uncertain': -0.5,
         'economic contraction':-1,
         'uncertainty arises':-0.5,
         'favorably expected signs':2,
         'consistent stabilization':1,
         'gradual recovery':1,
         'favorable global economic activity':1,
         'consistent recovery':1,
         'prospects economic recovery':1,
         'anchored expectations':1,
         'frustration expectations':-0.5,
         'evolved favorably':2,
         'benign evolution':2,
         'gradual normalization':1.5,
         'levels comfortable':2,
         'conditions favorable':2,
         'easing process continue':1,'employment gains':2, 'recovery advances':2, 'favorable evolution':1, 'consumption growth':2,
         'stronger recovery':2,'evolved expected':1,'lower global growth':-1,'benign behavior':1, 'comfortable levels':1
    
}

#Append weights to coefs from LR
for x,y in zip(list(dict_weights.keys()),list(dict_weights.values())):
    
    if x in list(coefs['word']):
        coefs.loc[coefs['word'] == x]['coef'].values[0] = y
    
    else:
        coefs = coefs.append({'coef':y,'word':x},ignore_index=True)
coefs

,coef,word
0,1.091135,index rose
1,0.636293,prime rate
2,0.518250,total return
3,0.391397,industrial average
4,0.381318,mutual fund
...,...,...
327,2.000000,stronger recovery
328,1.000000,evolved expected
329,-1.000000,lower global growth
330,1.000000,benign behavior


In [40]:
#Check pipeline again, so we can use it later on BCB texts
pipeline_ = Pipeline([

    ('tfidf', TfidfVectorizer(stop_words=stopw,strip_accents='unicode',ngram_range=(2,4),

    vocabulary = final_vocab)),

    ('clf',LogisticRegression(penalty='l1', C = 0.65))])


#New list with the coefs added above, with new weights included
list_coefs = np.array([coefs['coef'].to_list()])


score_nb=cross_val_score(pipeline_,X_train['text'],y_train,cv=StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42),scoring='f1_micro')

print(score_nb.mean())

print(score_nb.std())



0.5797191887675507
0.003638659528764625


In [41]:
# Fit do pipeline

pipeline_.fit(X_train['text'], y_train)

from sklearn.metrics import f1_score

print(f1_score(y_test, pipeline_.predict(X_test['text']), average = 'micro'))

0.5579937304075235


In [24]:
#Test with new coefs

#Setting new coefficients
pipeline_['clf'].coef_ = list_coefs

print(f1_score(y_test, pipeline_.predict(X_test['text']), average = 'micro'))

0.5538140020898642


In [25]:
#Checking LR coefs
pd.DataFrame({"coef":pipeline_['clf'].coef_[0],"word":pipeline_['tfidf'].get_feature_names()}).sort_values("coef", 
ascending=False).query("coef!=0")

,coef,word
321,2.000000,conditions favorable
309,2.000000,favorably expected signs
317,2.000000,evolved favorably
318,2.000000,benign evolution
327,2.000000,stronger recovery
326,2.000000,consumption growth
324,2.000000,recovery advances
323,2.000000,employment gains
320,2.000000,levels comfortable
319,1.500000,gradual normalization


In [33]:
#Testing in BCB's texts
#We are using a pre cleaned database with all the texts that matters for us - COPOM Minutes since 2016
d2 = pd.read_excel('cleaned_data.xlsx')
dv = pd.read_excel('base_val.xlsx')
d2 = d2.dropna()
d2.head()

,Unnamed: 0,Name,Text,EconOutlook,Risks_Scenario,MonetaryPol,MonetPolDec
0,0,copom 229 minutes,th update economic outlook global outlook nove...,global outlook novel coronavirus pandemic caus...,encompasses risk factors directions hand econo...,discussed evolution economic activity light av...,lower selic rate percentage judges decision re...
1,1,copom minutes 228,update economic outlook economic activity rele...,economic activity released previous indicate c...,encompasses risk factors directions hand high ...,discussed evolution economic activity light av...,lower selic rate percentage judges decision re...
2,2,copom minutes 230,update economic outlook scenario global outloo...,global outlook covid pandemic causing signific...,inflation encompasses risk factors directions ...,global economy judges previous international c...,lower selic rate percentage judges decision re...
3,3,copom_aug_5th,lowers selic rate lowers selic rate august pub...,following observations provide update global o...,prospectively uncertainty economic growth rema...,believes persevering process reforms necessary...,consequently possible future adjustments curre...
4,4,min2017204th-copom20170117-204th_copom_minutes,thcopommeetinga update economic outlook scenar...,set indicators released recently suggests weak...,risks inflation short run inflation behavior f...,discussed evolution economic activity light re...,reduce selic rate year bias judges convergence...


In [34]:
#Checking how the algorithm predicts the texts for us
#For the last minute, for example, we expect a negative sentiment, due to pandemic issues, etc
pipeline_.predict(d2['MonetaryPol'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0], dtype=int64)

In [39]:
pipeline_.predict_proba(d2['MonetaryPol'])[:,0]

array([0.61713158, 0.51781094, 0.52363385, 0.58325263, 0.52962619,
       0.58325263, 0.58325263, 0.59992158, 0.52647961, 0.3946541 ,
       0.20038519, 0.41402522, 0.52664025, 0.5714703 , 0.40125945,
       0.52377556, 0.47825334, 0.52132955, 0.45342591, 0.52437899,
       0.48182809, 0.56228385, 0.44275558, 0.54301707])

In [36]:
pipeline_.predict(d2['Risks_Scenario'])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)